In [1]:
!pip3 install gtts pydub langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=fa19e52c89be35a4c747840276adf050ef081a81df7095b66017971b7bfaa5bd
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [2]:
from gtts import gTTS
from pydub import AudioSegment
import math
import os
import time  # Import time module for sleep functionality

# Create a folder for audio files
audio_folder = 'audio_files'
os.makedirs(audio_folder, exist_ok=True)  # Create the folder if it doesn't exist

# Function to create speech file for a single word, its meaning, and examples
def create_speech_file(word, meaning, examples, file_index, lang='it'):
    # Create combined text
    full_text = f"{word} ({meaning})\n" + "\n".join(examples)

    # Generate audio
    tts = gTTS(text=full_text, lang=lang, slow=False)
    filename = os.path.join(audio_folder, f"word_{file_index}_{word}.mp3")  # Save in the audio folder
    tts.save(filename)
    print(f"Created {filename} with speech for {word}")
    return filename

# Function to check the duration of the audio file
def get_audio_duration(filename):
    audio = AudioSegment.from_file(filename)
    return len(audio) / 1000  # Duration in seconds

# Read the Italian words and their examples from the file
with open('/content/sentences.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Variables to store current word, meaning, and examples
current_word = ""
current_meaning = ""
current_examples = []
files_created = 0

# Process the lines to separate words, meanings, and examples
for line in lines:
    line = line.strip()
    if not line:
        continue  # Skip empty lines

    if '(' in line and ')' in line:  # Line contains a word and meaning
        if current_word:  # If there is a current word, create a speech file
            audio_filename = create_speech_file(current_word, current_meaning, current_examples, files_created + 1)
            duration = get_audio_duration(audio_filename)
            print(f"Duration of {audio_filename}: {duration:.2f} seconds")

            # Wait for a bit before the next request to avoid hitting the rate limit
            time.sleep(2)  # Adjust the time as needed (in seconds)

        # Reset for new word
        current_word = line.split('(')[0].strip()  # Get the word
        current_meaning = line.split('(')[1].strip(')')  # Get the meaning
        current_examples = []  # Reset examples list
        files_created += 1
    else:
        current_examples.append(line)  # Add example sentence

# Create audio for the last word after finishing the loop
if current_word:
    audio_filename = create_speech_file(current_word, current_meaning, current_examples, files_created + 1)
    duration = get_audio_duration(audio_filename)
    print(f"Duration of {audio_filename}: {duration:.2f} seconds")

print("All speech files generated!")

Created audio_files/word_2_il.mp3 with speech for il
Duration of audio_files/word_2_il.mp3: 27.98 seconds
Created audio_files/word_3_essere.mp3 with speech for essere
Duration of audio_files/word_3_essere.mp3: 27.29 seconds
Created audio_files/word_4_di.mp3 with speech for di
Duration of audio_files/word_4_di.mp3: 25.30 seconds
Created audio_files/word_5_uno.mp3 with speech for uno
Duration of audio_files/word_5_uno.mp3: 30.46 seconds
Created audio_files/word_6_e.mp3 with speech for e
Duration of audio_files/word_6_e.mp3: 30.12 seconds
Created audio_files/word_7_in.mp3 with speech for in
Duration of audio_files/word_7_in.mp3: 26.28 seconds
Created audio_files/word_8_a.mp3 with speech for a
Duration of audio_files/word_8_a.mp3: 28.44 seconds
Created audio_files/word_9_avere.mp3 with speech for avere
Duration of audio_files/word_9_avere.mp3: 26.98 seconds
Created audio_files/word_10_per.mp3 with speech for per
Duration of audio_files/word_10_per.mp3: 28.80 seconds
Created audio_files/wor

In [3]:
import zipfile
import os
def zip_audio_files(folder_name):
    zip_filename = f"{folder_name}.zip"  # Name of the zip file
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        # Walk through the folder and add files to the zip
        for root, dirs, files in os.walk(folder_name):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, folder_name))
    return zip_filename

In [4]:
from google.colab import files
zip_filename = zip_audio_files('audio_files')
print(f"Created zip file: {zip_filename}")
files.download(zip_filename)

Created zip file: audio_files.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
!pip install genanki

In [11]:
import genanki
import os
import random
import html

# Define a unique model_id and deck_id
model_id = random.randrange(1 << 30, 1 << 31)
deck_id = random.randrange(1 << 30, 1 << 31)

# Create the Anki model
my_model = genanki.Model(
    model_id,
    'Italian Vocabulary Model',
    fields=[
        {'name': 'Italian Word'},
        {'name': 'Meaning'},
        {'name': 'Examples'},
        {'name': 'Audio'},
    ],
    templates=[
        {
            'name': 'Card 1',
            'qfmt': '''
                <div class="card">
                    <div class="question">{{Italian Word}}</div>
                </div>
            ''',  # Front side: just the Italian word
            'afmt': '''
                <div class="card">
                    <div class="question">{{FrontSide}}</div>
                    <hr id="answer">
                    <div class="answer">
                        <div class="meaning">{{Meaning}}</div>
                        <div class="examples">
                            <ul>
                                {{Examples}}
                            </ul>
                        </div>
                        <div class="audio">{{Audio}}</div>
                    </div>
                </div>
            ''',  # Back side: Meaning, examples, and audio
        },
    ],
    css="""
        .card {
            font-family: 'Arial', sans-serif;
            font-size: 20px;
            text-align: center;
            color: #333;
            background-color: #f9f9f9;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
            margin: 20px;
        }
        .question {
            font-size: 36px;
            font-weight: bold;
            color: #2980b9;
            margin-bottom: 10px;
        }
        .answer {
            font-size: 20px;
            color: #555;
        }
        .meaning {
            font-size: 24px;
            margin: 10px 0;
            color: #2c3e50;
        }
        .examples {
            font-size: 18px;
            color: #7f8c8d;
            margin: 10px 0;
            list-style-type: none; /* Remove bullet points */
            padding: 0; /* Remove default padding */
        }
        .examples li {
            margin: 10px 0; /* Add margin between examples */
        }
        .examples li .translation {
            display: block;
            margin-left: 20px; /* Indent the translation */
            color: #34495e;
            font-style: italic;
        }
        .audio {
            font-size: 18px;
            margin-top: 20px;
        }
        a {
            color: #3498db;
            text-decoration: none;
        }
        a:hover {
            text-decoration: underline;
        }
    """
)

# Create a deck
my_deck = genanki.Deck(deck_id, 'Italian Vocabulary Deck')

# Prepare media files list
media_files = []

# Read the Italian words and their examples from the sentences.txt file
with open('/content/sentences.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

current_word = ""
current_meaning = ""
current_examples = []

# Process the lines to separate words, meanings, and examples
i = 0
while i < len(lines):
    line = lines[i].strip()
    if not line:
        i += 1
        continue  # Skip empty lines

    if '(' in line and ')' in line:  # Line contains a word and meaning
        if current_word:  # If there is a current word, create a note
            # Create the audio filename pattern based on the current word
            audio_filename = None
            for filename in os.listdir('/content/audio_files'):
                if filename.endswith(f"{current_word}.mp3"):
                    audio_filename = os.path.join('/content/audio_files', filename)
                    break

            # If audio is found, append media and add note
            if audio_filename and os.path.exists(audio_filename):
                media_files.append(audio_filename)
                # Create an Anki note
                my_note = genanki.Note(
                    model=my_model,
                    fields=[
                        html.escape(current_word),
                        html.escape(current_meaning),
                        "".join(f"<li>{html.escape(example[0])}<span class='translation'>{html.escape(example[1])}</span></li>"
                                 for example in current_examples),  # Format examples with translation under the Italian
                        f"[sound:{os.path.basename(audio_filename)}]",
                    ]
                )
                my_deck.add_note(my_note)

        # Reset for new word
        current_word = line.split('(')[0].strip()  # Get the word
        current_meaning = line.split('(')[1].strip(')')  # Get the meaning
        current_examples = []  # Reset examples list
        i += 1  # Move to next line
    else:
        # If it's an Italian sentence, the next line should be its English translation
        if i + 1 < len(lines):  # Ensure there's a next line
            italian_sentence = line
            english_translation = lines[i + 1].strip()  # Get the next line as translation
            current_examples.append((italian_sentence, english_translation))  # Add the pair
            i += 2  # Skip the next line (English translation)
        else:
            i += 1  # Just move to the next line if no translation is found

# Create an Anki note for the last word
if current_word:
    audio_filename = None
    for filename in os.listdir('/content/audio_files'):
        if filename.endswith(f"{current_word}.mp3"):
            audio_filename = os.path.join('/content/audio_files', filename)
            break

    if audio_filename and os.path.exists(audio_filename):
        media_files.append(audio_filename)
        my_note = genanki.Note(
            model=my_model,
            fields=[
                html.escape(current_word),
                html.escape(current_meaning),
                "".join(f"<li>{html.escape(example[0])}<span class='translation'>{html.escape(example[1])}</span></li>"
                         for example in current_examples),  # Format examples with translation under the Italian
                f"[sound:{os.path.basename(audio_filename)}]",
            ]
        )
        my_deck.add_note(my_note)

# Create an Anki package with media files
my_package = genanki.Package(my_deck)
my_package.media_files = media_files
my_package.write_to_file('italian_vocab_deck.apkg')  # Write the deck to a file

# Download the Anki deck
from google.colab import files
files.download('italian_vocab_deck.apkg')  # Download the generated Anki deck

print("Anki deck created successfully!")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Anki deck created successfully!
